In [3]:
!pip install nvcc4jupyter

In [4]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpzpvqvy0k".


In [6]:
%%cuda
#include<stdio.h>

__global__ void matMul(int *a, int *b, int *c, int width)
{
    int row = blockIdx.y*blockDim.y + threadIdx.y;
    int col = blockIdx.x*blockDim.x + threadIdx.x;


    if(row < width && col < width)
    {
        int sum=0;

        for(int i=0;i<width; i++)
        {
            sum+= a[row*width+i]*b[i*width+col];
        }

        c[row*width+col]=sum;
    }
}

void displayMatrix(int *c, int width)
{
    for(int i=0;i<width;i++)
    {
        for(int j=0;j<width;j++)
        {
            printf("%d\t", c[i*width+j]);
        }

        printf("\n");
    }
}

int main()
{
    const int width=3;

    const int size = width*width*sizeof(int);

    int h_mat1[width][width] = {{1, 2, 3}, {4, 5, 6}, {7, 8, 9}};

    int h_mat2[width][width] = {{9, 8, 7}, {6, 5, 4}, {3, 2, 1}};
    int h_result[width][width];

    int *d_mat1, *d_mat2, *d_result;

    cudaMalloc((void**)&d_mat1, size);
    cudaMalloc((void**)&d_mat2, size);
    cudaMalloc((void**)&d_result, size);

    cudaMemcpy(d_mat1, h_mat1, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_mat2, h_mat2, size, cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(16,16);
    dim3 blocksPerGrid((width+15)/16, (width+15)/16);

    matMul<<<blocksPerGrid,threadsPerBlock>>>(d_mat1, d_mat2, d_result, width);

    cudaMemcpy(h_result,d_result,size,cudaMemcpyDeviceToHost);

    displayMatrix((int *)h_result, width);

    cudaFree(d_mat1);
    cudaFree(d_mat2);
    cudaFree(d_result);

    return 0;

}


30	24	18	
84	69	54	
138	114	90	

